In [1]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime
import os
from dotenv import load_dotenv
from scipy.signal import find_peaks

print("All libraries imported successfully.")

load_dotenv()
mt5_account = int(os.getenv('MT5_ACCOUNT'))
mt5_password = os.getenv('MT5_PASSWORD')
mt5_server = os.getenv('MT5_SERVER')
mt5_path = os.getenv('MT5_PATH')

print("Credentials loaded.")

def find_swing_points(data, lookback_candles):
    swing_highs_indices, _ = find_peaks(data['high'], distance=lookback_candles)
    swing_lows_indices, _ = find_peaks(-data['low'], distance=lookback_candles)

    if len(swing_lows_indices) == 0 or len(swing_highs_indices) == 0:
        return None, None

    last_swing_high_time = data.index[swing_highs_indices[-1]]
    last_swing_low_time = data.index[swing_lows_indices[-1]]

    if last_swing_low_time < last_swing_high_time:
        return last_swing_low_time, last_swing_high_time
    else:
        if len(swing_lows_indices) > 1:
            prev_swing_low_time = data.index[swing_lows_indices[-2]]
            relevant_highs = swing_highs_indices[data.index[swing_highs_indices] > prev_swing_low_time]
            if len(relevant_highs) > 0:
                first_relevant_high_time = data.index[relevant_highs[0]]
                return prev_swing_low_time, first_relevant_high_time
    return None, None

print("Helper functions defined.")

def check_for_buy_signal(df_h4, df_h1):
    """
    Checks all strategy conditions to see if a buy signal is present.
    
    Args:
        df_h4 (pd.DataFrame): The H4 data with EMA calculated.
        df_h1 (pd.DataFrame): The H1 data with RSI calculated.
        
    Returns:
        dict or None: A dictionary with signal details if a buy is triggered, otherwise None.
    """
    print("\n--- Starting New Signal Check ---")
    
    last_h4_candle = df_h4.iloc[-1]
    is_uptrend = last_h4_candle['close'] > last_h4_candle['EMA_100']
    
    if not is_uptrend:
        print("Condition 1 FAILED: Price is not above the H4 100 EMA.")
        return None
    print(f"Condition 1 PASSED: Price ({last_h4_candle['close']:.3f}) is above H4 100 EMA ({last_h4_candle['EMA_100']:.3f}).")

    last_low_time, last_high_time = find_swing_points(df_h4, 10)
    
    if not (last_low_time and last_high_time):
        print("Condition 2 FAILED: Could not identify a valid swing structure.")
        return None

    swing_low_price = df_h4.loc[last_low_time]['low']
    swing_high_price = df_h4.loc[last_high_time]['high']
    price_range = swing_high_price - swing_low_price

    fibo_382 = swing_high_price - (price_range * 0.382)
    fibo_618 = swing_high_price - (price_range * 0.618)

    is_in_zone = fibo_618 <= last_h4_candle['close'] <= fibo_382

    if not is_in_zone:
        print(f"Condition 2 FAILED: Price ({last_h4_candle['close']:.3f}) is not in the Golden Zone ({fibo_618:.3f} - {fibo_382:.3f}).")
        return None
    print(f"Condition 2 PASSED: Price ({last_h4_candle['close']:.3f}) is in the Golden Zone ({fibo_618:.3f} - {fibo_382:.3f}).")

    last_h1_candle = df_h1.iloc[-1]
    prev_h1_candle = df_h1.iloc[-2]
    
    rsi_crossed_up = prev_h1_candle['RSI_14'] < 30 and last_h1_candle['RSI_14'] >= 30
    
    if not rsi_crossed_up:
        print(f"Condition 3 FAILED: H1 RSI ({last_h1_candle['RSI_14']:.2f}) did not just cross above 30.")
        return None
    print(f"Condition 3 PASSED: H1 RSI just crossed above 30 (from {prev_h1_candle['RSI_14']:.2f} to {last_h1_candle['RSI_14']:.2f}).")

    print("\n*** ALL CONDITIONS MET! ***")
    
    stop_loss = swing_low_price * 0.999 
    take_profit = swing_high_price + (price_range * 0.618)

    signal_details = {
        'symbol': 'XAUUSDm',
        'entry_price': last_h1_candle['close'],
        'stop_loss': stop_loss,
        'take_profit': take_profit,
        'reason': 'EMA Trend + Fibo Retracement + RSI Crossover'
    }
    
    return signal_details

print("Master signal function defined.")

if not mt5.initialize(path=mt5_path, login=mt5_account, password=mt5_password, server=mt5_server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()
print("\nExecution block: Connected to MT5.")

symbol = "XAUUSDm"
start_date = datetime(2023, 1, 1)
end_date = datetime.now()
df_h4 = pd.DataFrame(mt5.copy_rates_range(symbol, mt5.TIMEFRAME_H4, start_date, end_date))
df_h1 = pd.DataFrame(mt5.copy_rates_range(symbol, mt5.TIMEFRAME_H1, start_date, end_date))
df_h4['time'] = pd.to_datetime(df_h4['time'], unit='s')
df_h4.set_index('time', inplace=True)
df_h1['time'] = pd.to_datetime(df_h1['time'], unit='s')
df_h1.set_index('time', inplace=True)
print("Execution block: Fetched latest H4 and H1 data.")

df_h4['EMA_100'] = df_h4['close'].ewm(span=100, adjust=False).mean()
delta = df_h1['close'].diff()
gain = (delta.where(delta > 0, 0)).fillna(0)
loss = (-delta.where(delta < 0, 0)).fillna(0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df_h1['RSI_14'] = 100 - (100 / (1 + rs))
print("Execution block: Indicators calculated.")

signal = check_for_buy_signal(df_h4, df_h1)

if signal:
    print("\n✅✅✅ BUY SIGNAL GENERATED ✅✅✅")
    print(f"Symbol: {signal['symbol']}")
    print(f"Entry Price: {signal['entry_price']:.3f}")
    print(f"Stop Loss: {signal['stop_loss']:.3f}")
    print(f"Take Profit: {signal['take_profit']:.3f}")
    print(f"Reason: {signal['reason']}")
else:
    print("\n❌ No signal generated at this time. Conditions not met.")

mt5.shutdown()
print("\nExecution block: MT5 connection shut down.")

All libraries imported successfully.
Credentials loaded.
Helper functions defined.
Master signal function defined.

Execution block: Connected to MT5.
Execution block: Fetched latest H4 and H1 data.
Execution block: Indicators calculated.

--- Starting New Signal Check ---
Condition 1 PASSED: Price (3816.476) is above H4 100 EMA (3676.592).
Condition 2 FAILED: Price (3816.476) is not in the Golden Zone (3782.903 - 3797.021).

❌ No signal generated at this time. Conditions not met.

Execution block: MT5 connection shut down.
